# Hedge Fund Holdings Analysis via EDGAR

## 1. Setup and Imports

In [14]:
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup

import os
import re
import requests
import unicodedata
import pandas as pd
import csv
import xml.etree.ElementTree as ET
import yfinance as yf


In [5]:
dl = Downloader("Narain Nair", "nair4986@gmail.com")
dl.get("13F-HR", "1167483")

95

## 2. Parse and Structure Filing Data

In [6]:
def extract_info_table_xml(txt_file_path):
    """
    Extracts the XML string within <INFORMATIONTABLE> tags from the full .txt filing.
    """
    with open(txt_file_path, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()

    # Use regex to extract everything between <INFORMATIONTABLE>...</INFORMATIONTABLE>
    match = re.search(r"<INFORMATIONTABLE[\s\S]*?</INFORMATIONTABLE>", content, re.IGNORECASE)

    if match:
        return match.group(0)
    else:
        return None  # No XML found

In [7]:
def parse_info_table_xml(xml_string):
    """
    Parses the extracted XML string into a list of holdings as a pandas DataFrame.
    """
    xml_string_clean = re.sub(r'\sxmlns="[^"]+"', '', xml_string, count=1)

    root = ET.fromstring(xml_string_clean)

    data = []
    for info in root.findall("infoTable"):
        row = {}
        for child in info:
            if list(child):  # tag has nested children
                for subchild in child:
                    tag = subchild.tag.strip()
                    text = subchild.text.strip() if subchild.text else ''
                    row[tag] = text
            else:
                tag = child.tag.strip()
                text = child.text.strip() if child.text else ''
                row[tag] = text
        data.append(row)

    return pd.DataFrame(data)

In [8]:
def filter_2016(folder_name):
    match = re.search(r'-([0-9]{2})-', folder_name)
    if match:
        year_suffix = int(match.group(1))
        year = 2000 + year_suffix
        return year > 2016
    return False

In [21]:
def extract_date(accession):
    match = re.search(r'(\d{4})(\d{2})(\d{2})', accession)
    if match:
        return f"{match.group(1)}-{match.group(2)}-{match.group(3)}"
    else:
        return accession

In [23]:
## Constructing dictionaries for the holdings and the filings themselves
folder_path = "./sec-edgar-filings/0001167483/13F-HR/"
holdings = []
filings = []

for folder in os.listdir(folder_path):
    if not filter_2016(folder):
        continue

    access_folder_path = os.path.join(folder_path, folder)

    for file in os.listdir(access_folder_path):
        if file.endswith(".txt"):
            file_path = os.path.join(access_folder_path, file)
            print(f"Processing: {file_path}")
            
            xml_str = extract_info_table_xml(file_path)
            if xml_str:
                df = parse_info_table_xml(xml_str)
                df["source_file"] = file
                df["accession"] = folder

                date_str = extract_date(folder)
                filings.append({'date': date_str, 'df':df})
                holdings.append(df)

master_holdings_df = pd.concat(holdings, ignore_index=True)

Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-20-007148/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-24-004713/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-18-001706/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-22-006727/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-20-005337/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-18-005626/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-23-001481/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-21-007099/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-25-003217/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-19-001612/full-submission.txt
Processing: ./sec-edgar-filings/0001167483/13F-HR/0000919574-24-001349/full-submission.txt

## 3. Data processing & analysis

In [10]:
## Converting some of the columns to numeric datatypes.
cols_for_numeric = ['value', 'sshPrnamt', 'Sole', 'Shared', 'None']
for col in cols_for_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['%_allocation'] = 100 * (df['value'] / df['value'].sum())

In [11]:
## Function to extract the filing date from the path.
def filing_date_extraction(path):
    match = re.search(r'(\d{4}-\d{2}-\d{2}', path)
    if match:
        return match.group(1)
    else:
        return None

# 3.1 Portfolio-level metrics

In [19]:
## Calculating the allocation percentage per filing for each holding in portfolio
df = df.dropna(subset=['value'])
total_value = df['value'].sum()

df['allocation_percent'] = (df['value'] / total_value) * 100

In [27]:
def holdings_compare(df_1, df_2):
    prev_set = set(df_1['cusip'])
    curr_set = set(df_2['cusip'])

    new_buys = curr_set - prev_set
    full_exits = prev_set - curr_set
    kept = prev_set & curr_set

    return {
        'new_buys':len(new_buys),
        'exits':len(full_exits),
        'kept':len(kept),
        'total_prev':len(prev_set),
        'total_curr':len(curr_set),
        'turnover_pct':(len(new_buys | full_exits) / ((len(prev_set) + len(curr_set))/2)) * 100
    }

turnover_results = []
for i in range(1, len(filings)):
    prev = filings[i-1]
    curr = filings[i]
    comparison = holdings_compare(prev['df'], curr['df'])
    comparison['period'] = f"{prev['date']} -> {curr['date']}"
    turnover_results.append(comparison)

turnover_df = pd.DataFrame(turnover_results)